In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date
import seaborn as sns
import random

import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.mixture import GaussianMixture


from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

import scipy.stats as stats
from scipy.stats import probplot, laplace, norm, t


import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess

import pymc as pm
import pytensor.tensor as pt
import arviz as az

import tensorflow as tf
from tensorflow import keras

#from tensorflow.keras.utils import plot_model

######################################
#from pmdarima import auto_arima
#from diptest import diptest

In [ ]:
class StockPredictor:
    def __init__(self, ticker, start, end, column, nfuture = 252):

        self.ticker = ticker
        self.start = start
        self.end = end
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.nfuture = nfuture
        self.column=column
        self.rfr = 0.07

        self.df=None
        self.log_return = None
        self.df_hfd = None
        self.log_return_hfd = None

    def data_initiation_HFD(self):
        stock_data = StockData(self.ticker, self.start, self.end, self.column)
        self.df_hfd = stock_data.fetch_data_HFD()
        self.log_return_hfd = stock_data.daily_log_return_HFD()

    def data_initiation(self):
        stock_data = StockData(self.ticker, self.start, self.end, self.column)
        self.df = stock_data.fetch_data()
        self.log_return = stock_data.daily_log_return()

        expirations, self.strike, self.option_type ,self.obs_price, self.records = stock_data.download_option_data()
        start = pd.to_datetime(self.df.index[-1])
        expirations = pd.to_datetime(expirations)
        self.business_days = np.array([np.busday_count(start.date(), exp.date()) for exp in expirations])

    def run_implied(self, model_type):
        IV = ImpliedVolatility(self.df, self.log_return, self.strike, self.business_days * (1 / 252), self.option_type, self.obs_price, self.rfr)
        IV.compute_implied_vols()
        IV.plot_vol_smile()
        IV.plot_maturity()
        IV.plot_vol_surface()

        if model_type == 'dupire':
            K_vals = np.linspace(min(self.strike), max(self.strike), 25)
            T_vals = np.linspace(min(self.business_days * (1 / 252)), max(self.business_days * (1 / 252)), 25)
            IV.dupire_plot_local_vol(K_vals, T_vals)
        elif model_type == 'time':
            IV.time_plot_local_vol()
        else:
            raise ValueError("Unsupported model type")

    def run_ts(self, ar_order,  ma_order, sym_order, ema_smooth, nsim, alpha, model_type):
        window = 1 #arbitrary when intrday data not available

        #preprocess_data and check check_stationarity
        print ("CHECK STATIONARITY ----------------")
        StationarityCheck(self.df[self.column]).log_difference(ndiff = 1)
        print ("DONE : CHECK STATIONARITY ----------------")

        if model_type == 'EWMA':
            model =  EWMAVolatility(self.df_hfd, self.log_return_hfd, self.nfuture, window, ema_smooth)
            model.realized_volatility()
            model.forecast_volatility()
        elif model_type in ['ARCH', 'GARCH', 'TGARCH']:
            model = ARCHVolatility(self.df_hfd, self.log_return_hfd, model_type, self.nfuture, ar_order, ma_order, sym_order, window, ema_smooth)
            model.realized_volatility()
            model.forecast_volatility()
            print ('----- PARAMETER UNCERTAINTY----')
            model.forecast_volatility_uncertain(nsim, alpha)
        else:
            raise ValueError("Unsupported model type")

    def run_stochastic(self, nsim, alpha, model_type):
        if model_type == 'CIR':
            model = CIRModel(self.df_hfd, self.log_return_hfd, self.nfuture)

        model.realized_volatility()
        model.forecast_volatility(nsim, alpha)

    def run_ml(self, model_type):
        if model_type == 'NN':
            model = NN_RealizedVolatility(self.df_hfd, self.nfuture)
            model.realized_volatility()
            model.fit()
            model.forecast()





